In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdMolTransforms
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 300,300

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
import networkx as nx

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import SAGEConv, GCNConv, GATConv, GINConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import Data, Dataset, InMemoryDataset#, DataLoader depreciated, use below
from torch_geometric.loader import DataLoader

from torch_geometric.datasets import MoleculeNet, TUDataset
from torch_geometric.utils import to_networkx

from captum.attr import IntegratedGradients

In [ ]:
# Pytorch benchmark datasets
hiv_dataset = MoleculeNet(root = 'TorchDataset/', name = 'HIV')
clintox_dataset = MoleculeNet(root = 'TorchDataset/', name = 'ClinTox')
bbbp_dataset = MoleculeNet(root = 'TorchDataset/', name = 'BBBP')
bace_dataset = MoleculeNet(root = 'TorchDataset/', name = 'BACE')
tu_dataset = TUDataset(root = 'TorchDataset/', name = 'MUTAG')
esol_dataset = MoleculeNet(root = 'TorchDataset/', name = 'ESOL')

In [ ]:
#COMMON_ELEMENTS = {1, 6, 7, 8, 9, 15, 16, 17, 35, 53}
RDLogger.DisableLog('rdApp.*')

df_clintox = pytorch_to_df(clintox_dataset)
df_clintox = select_and_standardise(df_clintox, smiles_col='smiles', error_msg_col='error_msg', 
                       std_smiles_col='smiles_std')

df_clintox.to_csv('chem_data/clintox_standardised.csv')

In [ ]:
df_bbbp = pytorch_to_df(bbbp_dataset)
df_bbbp = select_and_standardise(df_bbbp, smiles_col = 'smiles', error_msg_col = 'error_msg',
                                 std_smiles_col = 'smiles_std')

df_bbbp.to_csv('chem_data/bbbp_standardised.csv')

In [ ]:
df_bace = pytorch_to_df(bace_dataset)
df_bace = select_and_standardise(df_bace, smiles_col = 'smiles', error_msg_col = 'error_msg',
                                 std_smiles_col = 'smiles_std')

df_bace.to_csv('chem_data/bace_standardised.csv')

#### Create custom pytorch dataset for clintox, bbbp and bace

In [ ]:
RDLogger.DisableLog('rdApp.*')

In [ ]:
## Prerun to make sure bad conformer ids (molecules) are identified
clintox_cus_prerun = MoleculeDataset(dataframe=df_clintox_filtered, smiles_col='smiles', 
                                  label_col='label', root='custom_data/clintox/prerun/')

error_indices = clintox_cus_prerun.error_indices  # bad_indices: Get the error indices
print(error_indices)
# df_clintox_filtered -> got rid of uncommon elements
# df_clintox_filtered_cleaned -> got rid of bad conformers
df_clintox_filtered_cleaned = df_clintox_filtered.drop(index=error_indices)  # bad_indices: Drop the error indices
df_clintox_filtered_cleaned.to_csv('chem_data/clintox_standardised_cleaned.csv')
print(df_clintox_filtered.shape, df_clintox_filtered_cleaned.shape)

In [ ]:
## We create two pytorch custom dataset: cus_2 -> effect of node + edge features
##                                   cus_2_std -> effect of standardisation of molecules
## Then we compare it to pytorch original dataset -> clintox_dataset_filtered

clintox_cus_2 = MoleculeDataset_updated(dataframe = df_clintox_filtered_cleaned,
                              smiles_col = 'smiles',
                             label_col = 'label',
                             root = 'custom_data/clintox/cus2/'
                            )
clintox_cus_2_std = MoleculeDataset_updated(dataframe = df_clintox_filtered_cleaned,
                              smiles_col = 'smiles_std',
                             label_col = 'label',
                             root = 'custom_data/clintox/cus2_std/'
                            )

## Original clintox dataset from pytorch
clintox_dataset = MoleculeNet(root = 'TorchDataset/', name = 'ClinTox')
# we will use FDA approval status as label
clintox_dataset.data.y = clintox_dataset.y[:,1]
# Select ones present after preprocessing (uncommon elements + bad conformers filtered)
filtered_smiles_set = set(df_clintox_filtered_cleaned['smiles']) # unique smiles
filtered_indices = [i for i, molecule in enumerate(clintox_dataset) if molecule['smiles'] in filtered_smiles_set]
# Filter the original dataset
clintox_dataset_filtered = clintox_dataset[filtered_indices]

# Change data type same as the custom dataset
clintox_dataset_filtered.data.y = clintox_dataset_filtered.data.y.to(torch.int64).unsqueeze(1) # float32 to int64
clintox_dataset_filtered.data.x = clintox_dataset_filtered.data.x.float() # int64 to float32
clintox_dataset_filtered.data.edge_attr = clintox_dataset_filtered.data.edge_attr.float() # int64 to float32

# Save filtered original pytorch dataset
torch.save(clintox_dataset_filtered, 'custom_data/clintox/pytorch/clintox_dataset_filtered.pt')
len(clintox_dataset_filtered)

#### Validate chemical structures after molecule standardisation and optimisation

In [ ]:
display(graph_to_mol(clintox_cus_2[1079]),
       graph_to_mol(clintox_cus_2_std[1079]),
       Chem.MolFromSmiles(clintox_dataset_filtered[1079].smiles))

#### Bond Length Analysis for DILI dataset

In [ ]:
def draw_2d_molecule(mol, title='Molecule'):
    drawer = rdMolDraw2D.MolDraw2DSVG(600, 600)  # Increased image size to 600x600
    drawer.drawOptions().addAtomIndices = True  # Show atom indices
    drawer.drawOptions().addStereoAnnotation = True  # Show stereo info
    rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    display(SVG(svg.replace('svg:', '')))

def get_bond_lengths(mol):
    conf = mol.GetConformer()
    bond_lengths = {}
    for bond in mol.GetBonds():
        start = bond.GetBeginAtomIdx()
        end = bond.GetEndAtomIdx()
        length = np.round(rdMolTransforms.GetBondLength(conf, start, end), 3)
        # Use atom indices and types for bond identification
        bond_key = (
            start, end,
            bond.GetBeginAtom().GetSymbol(), bond.GetEndAtom().GetSymbol()
        )
        bond_lengths[bond_key] = length
    return bond_lengths

def visualise_molecules():
    # Create a Penicillin V molecule
    penicillin_v_smiles = "CC1(C(N2C(S1)C(C2=O)NC(=O)COC3=CC=CC=C3)C(=O)O)C"  # Penicillin V (Phenoxymethylpenicillin)
    penicillin_v_mol = Chem.MolFromSmiles(penicillin_v_smiles)

    # Display 2D structure before optimisation
    print("Penicillin V Molecule (Before Optimisation):")
    draw_2d_molecule(penicillin_v_mol, title="Penicillin V (2D)")

    # Generate 3D coordinates for the initial structure
    penicillin_v_mol_initial = Chem.AddHs(penicillin_v_mol)
    AllChem.EmbedMolecule(penicillin_v_mol_initial, randomSeed=42, useRandomCoords=True, maxAttempts=5000)

    # Record bond lengths before optimisation
    initial_bond_lengths = get_bond_lengths(penicillin_v_mol_initial)

    # Optimise the molecule
    AllChem.MMFFOptimizeMolecule(penicillin_v_mol_initial)
    AllChem.ComputeGasteigerCharges(penicillin_v_mol_initial)

    # Record bond lengths after optimisation
    optimised_bond_lengths = get_bond_lengths(penicillin_v_mol_initial)

    # Remove hydrogens for drawing and consistency
    penicillin_v_mol_optimised = Chem.RemoveHs(penicillin_v_mol_initial)

    # Display 2D structure after optimisation
    print("\nPenicillin V Molecule (After Optimisation w/o standardisation):")
    draw_2d_molecule(penicillin_v_mol_optimised, title="Penicillin V Optimised (2D)")

    # Display Gasteiger charges for each atom
    print("\nGasteiger Charges:")
    for atom in penicillin_v_mol_initial.GetAtoms():  # Changed to initial molecule to ensure H atoms are present
        try:
            charge = atom.GetDoubleProp("_GasteigerCharge")
            print(f"Atom {atom.GetIdx()} ({atom.GetSymbol()}): Charge = {charge:.4f}")
        except KeyError:
            print(f"Atom {atom.GetIdx()} ({atom.GetSymbol()}): Charge not calculated")

    # Calculate bond length differences and percentage changes
    total_absolute_percentage_change = 0
    num_bonds = 0
    print("\nBond Length Differences and Percentage Changes (Initial - Optimised):")
    for bond_key, initial_length in initial_bond_lengths.items():
        if bond_key in optimised_bond_lengths:
            optimised_length = optimised_bond_lengths[bond_key]
            length_difference = np.round(initial_length - optimised_length, 3)
            percentage_change = (length_difference / initial_length) * 100 if initial_length != 0 else 0
            total_absolute_percentage_change += abs(percentage_change)
            num_bonds += 1
            print(f"Bond {bond_key[2]}{bond_key[0]}-{bond_key[3]}{bond_key[1]}: "
                  f"Initial = {initial_length} Å, Optimised = {optimised_length} Å, "
                  f"Difference = {length_difference} Å, "
                  f"Percentage Change = {percentage_change:.2f}%")

    # Calculate average absolute percentage change
    average_absolute_percentage_change = (total_absolute_percentage_change / num_bonds) if num_bonds > 0 else 0

    # Print total and average absolute percentage change
    #print(f"\nTotal Absolute Percentage Change: {total_absolute_percentage_change:.2f}%")
    print(f"Average Percentage Change per Bond: {average_absolute_percentage_change:.2f}%")

visualise_molecules()

In [ ]:
def get_bond_lengths(mol):
    conf = mol.GetConformer()
    bond_lengths = {}
    for bond in mol.GetBonds():
        start = bond.GetBeginAtomIdx()
        end = bond.GetEndAtomIdx()
        length = np.round(rdMolTransforms.GetBondLength(conf, start, end), 3)
        # Use atom indices and types for bond identification
        bond_key = (
            start, end,
            bond.GetBeginAtom().GetSymbol(), bond.GetEndAtom().GetSymbol()
        )
        bond_lengths[bond_key] = length
    return bond_lengths

def calculate_bond_length_changes(smiles):
    # Create a molecule from SMILES
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    # Generate 3D coordinates for the initial structure
    mol_initial = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol_initial, randomSeed=42, useRandomCoords=True, maxAttempts=5000)

    # Record bond lengths before optimisation
    initial_bond_lengths = get_bond_lengths(mol_initial)

    # Optimise the molecule
    AllChem.MMFFOptimizeMolecule(mol_initial)

    # Record bond lengths after optimisation
    optimised_bond_lengths = get_bond_lengths(mol_initial)

    # Calculate total and average absolute percentage change
    total_absolute_percentage_change = 0
    num_bonds = 0

    for bond_key, initial_length in initial_bond_lengths.items():
        if bond_key in optimised_bond_lengths:
            optimised_length = optimised_bond_lengths[bond_key]
            length_difference = np.round(initial_length - optimised_length, 3)
            percentage_change = (length_difference / initial_length) * 100 if initial_length != 0 else 0
            total_absolute_percentage_change += abs(percentage_change)
            num_bonds += 1

    # Calculate average absolute percentage change
    average_absolute_percentage_change = (total_absolute_percentage_change / num_bonds) if num_bonds > 0 else 0

    return total_absolute_percentage_change, average_absolute_percentage_change

In [ ]:
# Load the dataset
file_path = 'chem_data/DILI/DILIst_standardised_cleaned.csv'
df = pd.read_csv(file_path, index_col = 0)

# Calculate the changes for each molecule
for index, row in df.iterrows():
    smiles = row['smiles']  # Assuming the column name is 'smiles'
    total_change, average_change = calculate_bond_length_changes(smiles)
    #df.at[index, 'Total Percentage Change'] = total_change
    df.at[index, 'Bond Length Change per Bond'] = average_change

    smiles = row['smiles_std']
    total_change, average_change = calculate_bond_length_changes(smiles)
    #df.at[index, 'Total Percentage Change (std)'] = total_change
    df.at[index, 'Bond Length Change per Bond (std)'] = average_change

In [ ]:
df = df[['name', 'label', 'error_msg', 'smiles', 'Bond Length Change per Bond', 'smiles_std', 'Bond Length Change per Bond (std)']].reset_index(drop = True)
df.to_csv('chem_data/DILI/DILIst_bonds.csv')

In [ ]:
# Calculate basic statistics
normal_stats = df['Bond Length Change per Bond'].describe()
std_stats = df['Bond Length Change per Bond (std)'].describe()

normal_stats, std_stats

In [ ]:
from scipy.stats import ttest_rel

# Perform paired t-test
t_stat, p_value = ttest_rel(df['Bond Length Change per Bond'], df['Bond Length Change per Bond (std)'])

t_stat, p_value